In [36]:
!pip install -q -U transformers huggingface_hub datasets accelerate peft onnx onnxruntime optimum
# !pip install -q transformers==4.28.0 

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.9.2 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.2 which is incompatible.
tensorboard 2.9.1 requires protobuf<3.20,>=3.9.2, but you have protobuf 3.20.2 which is incompatible.


In [2]:
!curl -s https://packagecloud.io/install/repositories/github/git-lfs/script.deb.sh | sudo bash

Detected operating system as Ubuntu/focal.
Checking for curl...
Detected curl...
Checking for gpg...
Detected gpg...
Detected apt version as 2.0.9
Running apt-get update... done.
Installing apt-transport-https... done.
Installing /etc/apt/sources.list.d/github_git-lfs.list...done.
Importing packagecloud gpg key... Packagecloud gpg key imported to /etc/apt/keyrings/github_git-lfs-archive-keyring.gpg
done.
Running apt-get update... done.

The repository is setup! You can now install packages.


In [3]:
!sudo apt-get install git-lfs

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  git-lfs
0 upgraded, 1 newly installed, 0 to remove and 111 not upgraded.
Need to get 7419 kB of archives.
After this operation, 16.0 MB of additional disk space will be used.
Get:1 https://packagecloud.io/github/git-lfs/ubuntu focal/main amd64 git-lfs amd64 3.3.0 [7419 kB]
Fetched 7419 kB in 1s (11.9 MB/s)
Selecting previously unselected package git-lfs.
(Reading database ... 69943 files and directories currently installed.)
Preparing to unpack .../git-lfs_3.3.0_amd64.deb ...
Unpacking git-lfs (3.3.0) ...
Setting up git-lfs (3.3.0) ...
Git LFS initialized.
Processing triggers for man-db (2.9.1-1) ...


In [34]:
from huggingface_hub import notebook_login

notebook_login()

In [5]:
import torch
import transformers
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
import os
import matplotlib.pyplot as plt

os.environ["WANDB_DISABLED"] = "true"

tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

model = DistilBertForSequenceClassification.from_pretrained(
    "distilbert-base-uncased-finetuned-sst-2-english", 
    num_labels=1, 
    ignore_mismatched_sizes=True
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([2, 768]) in the checkpoint and torch.Size([1, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([2]) in the checkpoint and torch.Size([1]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [6]:
from datasets import load_dataset

dataset = load_dataset('ziq/ingredient_to_sugar_level')
dataset = dataset.rename_column("ingredients", "text").rename_column("sugar", "labels").remove_columns("src")

dataset = dataset.map(lambda x: tokenizer(x['text'], padding='max_length', truncation=True))
dataset

Found cached dataset parquet (/Users/ziqin/.cache/huggingface/datasets/ziq___parquet/ziq--ingredient_to_sugar_level-abc8bf7a5ca03ced/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec)


  0%|          | 0/2 [00:00<?, ?it/s]

Loading cached processed dataset at /Users/ziqin/.cache/huggingface/datasets/ziq___parquet/ziq--ingredient_to_sugar_level-abc8bf7a5ca03ced/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-5ab1971a06811368.arrow
Loading cached processed dataset at /Users/ziqin/.cache/huggingface/datasets/ziq___parquet/ziq--ingredient_to_sugar_level-abc8bf7a5ca03ced/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-d480b10eb64c2663.arrow


DatasetDict({
    train: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 22592
    })
    test: Dataset({
        features: ['text', 'labels', 'input_ids', 'attention_mask'],
        num_rows: 1965
    })
})

In [3]:
train_dataset = dataset['train'].shuffle(seed=42)
test_dataset = dataset['test'].shuffle(seed=42)
train_dataset, test_dataset

Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/ziq___parquet/ziq--ingredient_to_sugar_level-abc8bf7a5ca03ced/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-de7cab93b2c7c9b4.arrow
Loading cached shuffled indices for dataset at /root/.cache/huggingface/datasets/ziq___parquet/ziq--ingredient_to_sugar_level-abc8bf7a5ca03ced/0.0.0/2a3b91fbd88a2c90d1dbbb32b460cf621d31bd5b05b934492fdef7d8d6f236ec/cache-1f621f5d524f2847.arrow


(Dataset({
     features: ['text', 'labels', 'input_ids', 'attention_mask'],
     num_rows: 22592
 }),
 Dataset({
     features: ['text', 'labels', 'input_ids', 'attention_mask'],
     num_rows: 1965
 }))

In [4]:
BATCH_SIZE = 128
MICRO_BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = BATCH_SIZE // MICRO_BATCH_SIZE
LEARNING_RATE = 3e-4
TRAIN_STEPS = 2000
OUTPUT_DIR = "ingbetic"

In [5]:
trainer = transformers.Trainer(
    model=model,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    args=transformers.TrainingArguments(
        OUTPUT_DIR, 
        per_device_train_batch_size=MICRO_BATCH_SIZE,
        gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
        warmup_steps=100,
        max_steps=TRAIN_STEPS,
        learning_rate=LEARNING_RATE,
        fp16=True,
        logging_steps=10,
        evaluation_strategy="steps",
        save_strategy="steps",
        eval_steps=50,
        save_steps=50,
        save_total_limit=3,
        load_best_model_at_end=True,
        push_to_hub=True,
    ),
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).
/notebooks/ingbetic is already a clone of https://huggingface.co/ziq/ingbetic. Make sure you pull the latest changes with `repo.git_pull()`.


In [4]:
# trainer.push_to_hub()

In [ ]:
trainer.train("ingbetic/checkpoint-1500")

In [8]:
trainer.push_to_hub()

Upload file pytorch_model.bin:   0%|          | 1.00/255M [00:00<?, ?B/s]

Upload file runs/Jun07_18-30-10_ny736lt2rd/events.out.tfevents.1686162626.ny736lt2rd.9572.0:   0%|          | …

To https://huggingface.co/ziq/ingbetic
   a5eb3e7..4708bbb  main -> main

To https://huggingface.co/ziq/ingbetic
   4708bbb..60450a0  main -> main



'https://huggingface.co/ziq/ingbetic/commit/4708bbb8b6ee82ff9514601c63b487ad9c8d93b5'

In [7]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained("ziq/ingbetic")     

In [14]:
dummy = tokenizer("1 pound ground beef, 1 tablespoon dehydrated onion, 1 teaspoon salt, 1/2 teaspoon pepper, 1 teaspoon garlic powder, 1/4 teaspoon oregano, 1 cup uncooked penne pasta, 1 cup water, 1 (15-ounce) can fire-roasted diced tomatoes, 1/4 cup Parmesan cheese, 1/2 cup shredded mozzarella cheese, 4-6 fresh basil leaves, torn", padding='max_length', truncation=True)
input_ids = torch.tensor(dummy['input_ids']).unsqueeze(0)
attention_mask = torch.tensor(dummy['attention_mask']).unsqueeze(0)
input_ids.shape

torch.Size([1, 512])

In [24]:
import numpy as np

x = torch.tensor([dummy['input_ids']])

In [25]:
model(input_ids=x)

SequenceClassifierOutput(loss=None, logits=tensor([[2.7034]], grad_fn=<AddmmBackward0>), hidden_states=None, attentions=None)

In [14]:
model(input_ids=input_ids, attention_mask=attention_mask)['logits']

tensor([[-0.4866]], grad_fn=<AddmmBackward0>)

In [10]:
pipeline = transformers.pipeline("text-classification",model=model,tokenizer=tokenizer)

Xformers is not installed correctly. If you want to use memorry_efficient_attention to accelerate training use the following command to install Xformers
pip install xformers.


In [11]:
pipeline("1 pound ground beef, 1 tablespoon dehydrated onion, 1 teaspoon salt, 1/2 teaspoon pepper, 1 teaspoon garlic powder, 1/4 teaspoon oregano, 1 cup uncooked penne pasta, 1 cup water, 1 (15-ounce) can fire-roasted diced tomatoes, 1/4 cup Parmesan cheese, 1/2 cup shredded mozzarella cheese, 4-6 fresh basil leaves, torn")

[{'label': 'LABEL_0', 'score': 0.38068893551826477}]

In [34]:
torch.onnx.export(
    model, 
    tuple((input_ids, attention_mask)),
    f="sugar.onnx",  
    input_names=['input_ids', 'attention_mask'], 
    output_names=['logits'], 
    dynamic_axes={'input_ids': {0: 'batch_size', 1: 'sequence'}, 
                  'attention_mask': {0: 'batch_size', 1: 'sequence'}, 
                  'logits': {0: 'batch_size', 1: 'sequence'}}, 
    do_constant_folding=True, 
    opset_version=13, 
)

In [33]:
from onnxruntime.quantization import quantize_dynamic, QuantType
quantize_dynamic("sugar.onnx", "sugar-int8.onnx", 
                 weight_type=QuantType.QUInt8)

Ignore MatMul due to non constant B: /[/distilbert/transformer/layer.0/attention/MatMul]
Ignore MatMul due to non constant B: /[/distilbert/transformer/layer.0/attention/MatMul_1]
Ignore MatMul due to non constant B: /[/distilbert/transformer/layer.1/attention/MatMul]
Ignore MatMul due to non constant B: /[/distilbert/transformer/layer.1/attention/MatMul_1]
Ignore MatMul due to non constant B: /[/distilbert/transformer/layer.2/attention/MatMul]
Ignore MatMul due to non constant B: /[/distilbert/transformer/layer.2/attention/MatMul_1]
Ignore MatMul due to non constant B: /[/distilbert/transformer/layer.3/attention/MatMul]
Ignore MatMul due to non constant B: /[/distilbert/transformer/layer.3/attention/MatMul_1]
Ignore MatMul due to non constant B: /[/distilbert/transformer/layer.4/attention/MatMul]
Ignore MatMul due to non constant B: /[/distilbert/transformer/layer.4/attention/MatMul_1]
Ignore MatMul due to non constant B: /[/distilbert/transformer/layer.5/attention/MatMul]
Ignore MatM

In [39]:
# from transformers import AutoTokenizer
# from optimum.onnxruntime import ORTModelForSequenceClassification

# # load model from hub and convert
# model = ORTModelForSequenceClassification.from_pretrained("ziq/ingbetic",from_transformers=True)
# tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased-finetuned-sst-2-english")

# # save converted model
# model.save_pretrained("ingbetic")
# tokenizer.save_pretrained("ingbetic")

In [50]:
import onnx
onnx_model = onnx.load("sugar.onnx")
onnx.checker.check_model(onnx_model)

In [2]:
import onnxruntime as ort
import numpy as np
ort_sess = ort.InferenceSession('model-int8.onnx')

In [9]:
input_ids = dummy['input_ids']
attention_mask = dummy['attention_mask']

In [17]:
outputs = ort_sess.run(None, {'input_ids': input_ids.numpy(), 'attention_mask': attention_mask.numpy()})

# Print Result 
outputs[0] #* 13.3627190349059 + 10.85810766787474

array([[-0.47709924]], dtype=float32)

In [ ]:
from huggingface_hub import HfApi
api = HfApi()

# api.upload_file(
#     path_or_fileobj="sugar.onnx",
#     path_in_repo="model.onnx",
#     repo_id="ziq/ingbetic",
#     repo_type="model",
# )

# api.upload_file(
#     path_or_fileobj="sugar-int8.onnx",
#     path_in_repo="model-int8.onnx",
#     repo_id="ziq/ingbetic",
#     repo_type="model",
# )

In [ ]:
api.upload_file(
    path_or_fileobj="main.ipynb",
    path_in_repo="train.ipynb",
    repo_id="ziq/ingbetic",
    repo_type="model",
)